**해야할 것**

---
1. train셋, validation셋 나눠서 스코어 출력하기
2. 5-fold cross validation으로 스코어 출력하기
3. 5-fold cross validation에서 grid search하기
---
기존에는 GridsearchCV 패키지를 이용해 1,2,3번에 해당하는 작업을 편하게 진행했지만, INPUT에 정형데이터가 추가되어 더이상 GridsearchCV에 의존할 수 없다.

이 노트북에서는 아직 1번까지 달성.
2,3 번 빨리해야함.

In [ ]:
!pip install torchaudio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from pathlib import Path
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torchvision
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from sklearn.metrics import confusion_matrix
import numpy as np
import sklearn.metrics as metrics

In [ ]:
Catholic_file = '/content/drive/MyDrive/breath_data/breath_v2.2/aug_train_v2.2.csv'
df = pd.read_csv(Catholic_file)
df

,filename,category,class,sex,old
0,0002-1.wav,non-wheezing,0,1,7
1,0002-2.wav,non-wheezing,0,1,7
2,0002-3.wav,non-wheezing,0,1,7
3,0002-4.wav,non-wheezing,0,1,7
4,0003-2.wav,non-wheezing,0,0,1
...,...,...,...,...,...
1605,wn_0535-4.wav,wheezing,1,0,9
1606,wn_0588-1.wav,wheezing,1,0,1
1607,wn_0599-3.wav,wheezing,1,1,6
1608,wn_0602-2.wav,wheezing,1,1,8


In [ ]:
mean = df.old.mean()
std = df.old.std()
df.old = (df.old - mean) / std

In [ ]:
df

,filename,category,class,sex,old
0,0002-1.wav,non-wheezing,0,1,0.905217
1,0002-2.wav,non-wheezing,0,1,0.905217
2,0002-3.wav,non-wheezing,0,1,0.905217
3,0002-4.wav,non-wheezing,0,1,0.905217
4,0003-2.wav,non-wheezing,0,0,-0.948195
...,...,...,...,...,...
1605,wn_0535-4.wav,wheezing,1,0,1.523021
1606,wn_0588-1.wav,wheezing,1,0,-0.948195
1607,wn_0599-3.wav,wheezing,1,1,0.596315
1608,wn_0602-2.wav,wheezing,1,1,1.214119


In [ ]:
df['relative_path'] = '/' + df['filename'].astype(str)
df = df[['relative_path', 'class', 'sex', 'old']]
df.head()

,relative_path,class,sex,old
0,/0002-1.wav,0,1,0.905217
1,/0002-2.wav,0,1,0.905217
2,/0002-3.wav,0,1,0.905217
3,/0002-4.wav,0,1,0.905217
4,/0003-2.wav,0,0,-0.948195


In [ ]:
data_path = '/content/drive/MyDrive/breath_data/breath_v2.2/aug_train_v2.2'

In [ ]:
class Breath_sound_Util():
  
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    
    return (sig, sr)

  def resample(aud, newsr):
    sig, sr = aud
    
    if (sr == newsr):
     return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))
  

  def pad(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms
   
    if (sig_len > max_len):
      sig = sig[:,:max_len]
 
    elif (sig_len < max_len):

      repeated = []
      repeated.append(sig)
      required_len = max_len - sig_len

      while required_len > sig_len : 
        repeated.append(sig)
        require_len -= sig_len
      repeated.append(sig[:, :required_len])
 
      sig = torch.cat(repeated, 1)

    return (sig, sr)


  def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80
    
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)

  def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
    _, n_mels, n_steps = spec.shape
    mask_value = spec.mean()
    aug_spec = spec

    freq_mask_param = max_mask_pct * n_mels
    for _ in range(n_freq_masks):
      aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

    time_mask_param = max_mask_pct * n_steps
    for _ in range(n_time_masks):
      aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

    return aug_spec
    print(aug_spec)

In [ ]:
class breathDS(Dataset):
    
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 48000
            
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    class_id = self.df.loc[idx, 'class']
    
    aud = Breath_sound_Util.open(audio_file)
    reaud = Breath_sound_Util.resample(aud, self.sr)
    dur_aud = Breath_sound_Util.pad(reaud, self.duration)
    sgram = Breath_sound_Util.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = Breath_sound_Util.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
    #여기부터 나이, 성별변수를 집어넣는 과정임
    x = self.df.loc[idx, 'old']
    y = self.df.loc[idx, 'sex']
    x = torch.from_numpy(np.asarray(x).reshape((1,)))
    y = torch.from_numpy(np.asarray(y).reshape((1,)))
    tabular = torch.cat((x, y), 0)
    tabular = tabular.float()
    #print(tabular)
    #x1 = ["sex", "old"]
    #x2 = x2.
    #x2 = x2.iloc[idx].values
    
    
    return aug_sgram, tabular, class_id

In [ ]:
brds = breathDS(df, data_path)

In [ ]:
num_items = len(brds)
print(len(brds))
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(brds, [num_train, num_val])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=True)

1610


In [ ]:
train_dl = DataLoader(brds, batch_size=16, shuffle=True)

In [ ]:
# --------------------------------------------------------------
# 호흡음의 Healthy, Wheezing을 판단하는 Binary Classification Model
# --------------------------------------------------------------


class WhoWheezing(nn.Module):
    

    def __init__(self):
        super().__init__()
        conv_layers = []
        self.conv1 = nn.Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv1.weight, a=0.2)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.dropout = nn.Dropout(0.4)
        self.lin = nn.Linear(in_features=72, out_features=2)
        self.dropout = nn.Dropout(0.4)
        self.sigmoid = nn.Sigmoid()

        self.conv = nn.Sequential(*conv_layers)
 
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 8)

    def forward(self, inputs, inputs2):
        inputs = self.conv(inputs)
        inputs = self.ap(inputs)
        inputs = inputs.view(inputs.shape[0], -1)
        inputs2 = self.fc1(inputs2)
        inputs2 = self.fc2(inputs2)
        #inputs가 원래있던 멜스펙트로그램이고
        #inputs2가 정형데이터임
        x = torch.cat((inputs, inputs2), 1)
        x = self.lin(x)
        x = self.dropout(x)
        inputs = self.sigmoid(x)
        return x       

    '''def forward(self, x):
        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        #print(x.size())
        #x2 = category
        #x2 = x2(self.fc1(x2))
        #x2 = x2(self.fc2(x2))
        #x = x1 + x2
        x = self.lin(x)
        x = self.dropout(x)
        x = self.sigmoid(x)
        return x'''
    
Model1 = WhoWheezing()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Model1 = Model1.to(device)
next(Model1.parameters()).device

device(type='cuda', index=0)

In [ ]:
def training(model, train_dl, val_dl, num_epochs):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0
    correct_prediction_1 = 0
    total_prediction_1 = 0

    for i, data in enumerate(train_dl):
        inputs, inputs2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        #여기에도 inputs2를 추가시켜줫음. 이제 data[1]이 가리키는건 breathDS의 tablular임
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s
        optimizer.zero_grad()
        outputs = model(inputs, inputs2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
        _, prediction = torch.max(outputs,1)
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

    for i, data in enumerate(val_dl):
        inputs, inputs2, labels = data[0].to(device), data[1].to(device), data[2].to(device)
        #여기에도 inputs2를 추가시켜줫음. 이제 data[1]이 가리키는건 breathDS의 tablular임
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s
        outputs = model(inputs, inputs2)
        _, prediction = torch.max(outputs,1)
        correct_prediction_1 += (prediction == labels).sum().item()
        total_prediction_1 += prediction.shape[0]

    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    val_acc = correct_prediction_1/total_prediction_1
    print(f'Epoch: {epoch}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}, Validation Accuracy: {val_acc:.4f}')

  print('Finished Training')

In [ ]:
num_epochs=52
training(Model1, train_dl, val_dl, num_epochs)

Epoch: 0, Loss: 0.4910, Accuracy: 0.7943, Validation Accuracy: 0.7981
Epoch: 1, Loss: 0.4853, Accuracy: 0.8106, Validation Accuracy: 0.7826
Epoch: 2, Loss: 0.4671, Accuracy: 0.8253, Validation Accuracy: 0.8075
Epoch: 3, Loss: 0.4457, Accuracy: 0.8191, Validation Accuracy: 0.8043
Epoch: 4, Loss: 0.4686, Accuracy: 0.8152, Validation Accuracy: 0.8106
Epoch: 5, Loss: 0.4510, Accuracy: 0.8168, Validation Accuracy: 0.8168
Epoch: 6, Loss: 0.4386, Accuracy: 0.8230, Validation Accuracy: 0.7950
Epoch: 7, Loss: 0.4480, Accuracy: 0.8175, Validation Accuracy: 0.8168
Epoch: 8, Loss: 0.4508, Accuracy: 0.8245, Validation Accuracy: 0.8012
Epoch: 9, Loss: 0.4553, Accuracy: 0.7958, Validation Accuracy: 0.8137
Epoch: 10, Loss: 0.4526, Accuracy: 0.8175, Validation Accuracy: 0.8261
Epoch: 11, Loss: 0.4181, Accuracy: 0.8377, Validation Accuracy: 0.8323
Epoch: 12, Loss: 0.4249, Accuracy: 0.8284, Validation Accuracy: 0.8075
Epoch: 13, Loss: 0.4433, Accuracy: 0.8276, Validation Accuracy: 0.8012
Epoch: 14, Loss: